# Task B: Solve Dinner Planning with CSP

## Problem Description

"The Dupont family will visit us tonight," Mr. Martin announces. "The whole family, that is, Mr. and
Mrs. Dupont and their three children Emma, Georg and Ivana?" Mrs. Martin asks in dismay.
Mr. Martin replies: "That's the problem: they wanted to make a secret about who exactly wants to come. 
But we know: If Mr. Dupont comes, then his wife will come, too.
At least one of the two children Ivana and Georg will come. Either Mrs. Dupont or Emma will come.
Either Emma and Georg come both, or they both don't come. And if Ivana comes, then also Georg and
Mr. Dupont will come."

Help the Martins with the dinner planning: Who will visit the Martins tonight?

## Abbreviations
```
Mr H
Mrs W
Emma E
Georg G
Ivana I

-> H W E G I
```

## Constrains
```
H -> W
I or G
W xor E
E <-> G
I -> (G and H)
```

## Constrains in CNF
```
(-H or W)        -> ["-H", "W"]
(I or G)         -> ["I", "G"]
(W or E)         -> ["W", "E"]
(not W or not E) -> ["-W", "-E"]
(-E or G)        -> ["-E", "G"]
(-G or E)        -> ["-G", "E"]
(-I or G)        -> ["-I", "G"]
(-I or H)        -> ["-I", "H"]
```

In [220]:
import copy

def neg(var: str):
    if var[0] == "-":
        return var[1:]
    else:
        return "-" + var
    
def is_neg(var: str):
    return var[0] == "-"
    
assert("A" == neg("-A"))
assert("-A" == neg("A"))
assert(True == is_neg("-A"))
assert(False == is_neg("A"))

def unique(l: list[str]):
    result = list(set(l))
    result.sort()
    return result

def sort_array(l: list[str]):
    result = copy.deepcopy(l)
    result.sort()
    return result

assert(["A", "B", "C"] == unique(["A", "B", "C", "A", "B", "C"]))

In [221]:
def remove_unit_clauses(result, clauses):
    while True:
        unitClauses = []
        for clause in clauses:
            if len(clause) == 1 and clause[0] not in unitClauses:
                unitClauses.append(clause[0])
        if len(unitClauses) == 0:
            break

        # remove all clauses that contain unit clauses (literals)
        for (i, clause) in enumerate(clauses):
            for unitClause in unitClauses:
                if unitClause in clause:
                    clauses[i] = None
        clauses = [clause for clause in clauses if clause is not None] # Delete the 

        # remove all negated unit clauses (literals) from all clauses
        for (i, clause) in enumerate(clauses):
            for unitClause in unitClauses:
                if neg(unitClause) in clause:
                    clauses[i].remove(neg(unitClause))

        # add the unitClauses to the result
        for unitClause in unitClauses:

            # if unitClause is already negated in the result, then there is no solution -> return empty result
            if neg(unitClause) in result:
                return [], clauses
            
            # if unitClause is not already in the result, then add it
            if unitClause not in result:
                result.append(unitClause)

    return unique(result), clauses

In [222]:
def simple_csp_solver(result: list[str], clauses: list[list[str]]):

    # first remove all unit clauses
    result, clauses = remove_unit_clauses(result, clauses)
    
    # if null clause is present, return an empty list
    for clause in clauses:
        if len(clause) == 0:
            return []
        
    # if no clauses are left, return the result
    if len(clauses) == 0:
        return result
    
    # get a literal and build two new clauses
    literal = clauses[0][0]
    clauses_1 = clauses + [[literal]]
    clauses_2 = clauses + [[neg(literal)]]

    # call recursively with the two new clauses
    result_1 = simple_csp_solver(result, clauses_1)
    if len(result_1) > 0:
        return result_1
    return simple_csp_solver(result, clauses_2)

# solve the dinner planning problem
constrains = [  # see markdown cell
    ["-H", "W"],
    ["I", "G"],
    ["W", "E"],
    ["-W", "-E"],
    ["-E", "G"],
    ["-G", "E"],
    ["-I", "G"],
    ["-I", "H"],
]
should_solution = ["-H", "-W", "-I", "G", "E"] # found by brute force
is_solution = simple_csp_solver([], constrains)
print("should_solution: ", should_solution)
print("is_solution: ", is_solution)
print("is_solution == should_solution: ", sort_array(should_solution) == sort_array(is_solution))
assert(sort_array(should_solution) == sort_array(is_solution))

should_solution:  ['-H', '-W', '-I', 'G', 'E']
is_solution:  ['-H', '-I', '-W', 'E', 'G']
is_solution == should_solution:  True
